In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#train 데이터
# CSV파일에서 데이터를 불러온다
# 뒤의 1시간동안 1% 상승하면 1  (수수료가 0.04%인것 감안)
# 그외에는하락하면 0으로 라벨링
x_data = {}
y_labels = {}
for num in range(24):
    xy = np.loadtxt('train_'+str(num)+'.csv', delimiter=',', dtype = np.float32)
    x_data['train_'+str(num)] = xy[:, :48]/10000000

y_data =xy[:, 48:]
y_label = []
for i in range(np.shape(xy)[0]):
    if ((xy[i, 50] - xy[i, 48])/xy[i, 48]) > 0.01:
        y_label.append(1)
    else:
        y_label.append(0)
y_label = np.reshape(y_label, [np.shape(xy)[0],1])
y_labels['train_'+str(num)] = y_label

In [3]:
#test 데이터

xy_test = np.loadtxt('train_24.csv', delimiter=',', dtype = np.float32)
x_test_data = xy_test[:, :48]
print(np.shape(x_test_data))
y_test_data =xy_test[:, 48:]
y_test_label = []
for i in range(np.shape(xy_test)[0]):
    if (xy_test[i, 50] - xy_test[i, 48]) > 0:
        y_test_label.append(1)
    else:
        y_test_label.append(0)
y_test_label = np.reshape(y_test_label, [np.shape(xy_test)[0],1])

(240, 48)


In [4]:
x = tf.placeholder(tf.float32, shape = [None, 48])
y = tf.placeholder(tf.float32, shape = [None, 1])

w1 = tf.get_variable('weight1', shape=[48,256], initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name = 'bias1')
_layer1 = tf.nn.relu(tf.matmul(x,w1) + b1)

w2 = tf.get_variable('weight2', shape=[256,128], initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([128]), name = 'bias2')
_layer2 = tf.nn.relu(tf.matmul(_layer1,w2) + b2)

w3 = tf.get_variable('weight3', shape=[128,64], initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([64]), name = 'bias3')
_layer3 = tf.nn.relu(tf.matmul(_layer2,w3) + b3)

w4 = tf.get_variable('weight4', shape=[64,1], initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1]), name = 'bias4')
hypothesis = tf.sigmoid(tf.matmul(_layer3,w4) + b4)
cost = -tf.reduce_mean(y*tf.log(hypothesis) + (1-y)*tf.log(1-hypothesis))


optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train = optimizer.minimize(cost)

predict = tf.cast(hypothesis > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict,y), dtype = tf.float32))

training_epochs = 1001

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = 24
    
    for i in range(total_batch):
        c, _ = sess.run([cost, train], feed_dict = {x: x_data['train_'+str(num)], y: y_labels['train_'+str(num)]})
        avg_cost += c /total_batch
        
    if epoch % 100 == 0:
        print('Epoch:', '%04d' % (epoch +1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Accuracy:', accuracy.eval(session=sess, feed_dict = {x: x_test_data, y: y_test_label}))
print('Accuracy:', accuracy.eval(session=sess, feed_dict = {x: x_data['train_0'], y: y_labels['train_0']}))

Epoch: 0001 cost = 0.006179588
Epoch: 0101 cost = 0.000000000
Epoch: 0201 cost = 0.000000000
Epoch: 0301 cost = 0.000000000
Epoch: 0401 cost = 0.000000000
Epoch: 0501 cost = 0.000000000
Epoch: 0601 cost = 0.000000000
Epoch: 0701 cost = 0.000000000
Epoch: 0801 cost = 0.000000000
Epoch: 0901 cost = 0.000000000
Epoch: 1001 cost = 0.000000000
Accuracy: 0.5208333


KeyError: 'train_0'